In [30]:
import os
import re
import importlib
import time

from datetime import datetime

In [31]:
import math

def degree_to_radian(degrees):
    '''Converts degrees (in decimal) to radians.'''
    return (math.pi/180)*degrees

def radian_to_nm(radians):
    '''Converts a distance in radians to a distance in nautical miles.'''
    return ((180*60)/math.pi)*radians

def calc_dist(latitude1,longitude1,latitude2,longitude2):
    '''Calculates and returns a distance in nautical miles given two lat/lon coordinate values.
        Follows the specification described in the Aviation Formulary v1.46
        by Ed Williams (originally at http://williams.best.vwh.net/avform.htm)
    '''
    lat1 = degree_to_radian(latitude1)
    lon1 = degree_to_radian(longitude1)
    lat2 = degree_to_radian(latitude2)
    lon2 = degree_to_radian(longitude2)

    # there are two implementations of this function.
    # implementation #1:
    #dist_rad = math.acos(math.sin(lat1) * math.sin(lat2) 
    #                   + math.cos(lat1) * math.cos(lat2) * math.cos(lon1-lon2))

    # implementation #2: (less subject to numerical error for short distances)
    dist_rad=2*math.asin(math.sqrt((math.sin((lat1-lat2)/2))**2 +
               math.cos(lat1)*math.cos(lat2)*(math.sin((lon1-lon2)/2))**2))

    return radian_to_nm(dist_rad)

In [32]:

filename = 'Agadir'

filepath = os.path.join("cfg", filename + '.py')

instance = importlib.import_module("cfg." + filename)

start_time = time.time()

In [33]:
print ("reading '" + instance.DIR + instance.INPUT + "'")

file = open(instance.DIR + "/" + instance.INPUT, "r")
elevation_data = file.read()
file.close()

ncols = int(re.search("ncols (.*)", elevation_data).group(1))
print ("number of columns:",ncols)

nrows = int(re.search("nrows (.*)", elevation_data).group(1))
print ("number of rows:", nrows)

latitude = float(re.search("xllcorner (.*)", elevation_data).group(1))
longitude = float(re.search("yllcorner (.*)", elevation_data).group(1))
data_delta = float(re.search("cellsize (.*)", elevation_data).group(1))

elevation_data = re.split("\n+", elevation_data)[6:-1]

print(len(elevation_data))

if (nrows != len(elevation_data)+1):

	print ("something wrong here")
	#quit()

map = {}
ocean = {}
min_depth = -11022.0
max_depth = 0.0

for i, line_str in enumerate(elevation_data[(nrows - 100 - instance.Y):nrows - 100]):

    line_dict = {}
	
    for j, element in enumerate(re.split("\s+", line_str)[:instance.X]):

        # if needed, here has to be the code for avg of depths

        if float(element) < 0:

            ocean[i,j] = 1

            if float(element) > min_depth and float(element) < 0:

                min_depth = float(element)

            if float(element) < max_depth:

                max_depth = float(element)

            line_dict[j] = float(element)

    map[i] = line_dict

reading 'Instances/Agadir/GMRTv3_4_20171109topo.asc'
number of columns: 696
number of rows: 609
608


In [34]:
map

{0: {0: -93.1,
  1: -93.24,
  2: -93.4,
  3: -93.41,
  4: -93.1,
  5: -92.63,
  6: -92.1,
  7: -91.54,
  8: -91.07,
  9: -90.6,
  10: -89.99,
  11: -89.47},
 1: {0: -91.48,
  1: -91.85,
  2: -92.1,
  3: -92.21,
  4: -92.14,
  5: -91.94,
  6: -91.63,
  7: -91.23,
  8: -90.8,
  9: -90.29,
  10: -89.59,
  11: -88.99},
 2: {0: -89.5,
  1: -90.03,
  2: -90.51,
  3: -90.9,
  4: -91.08,
  5: -91.13,
  6: -91.1,
  7: -90.93,
  8: -90.57,
  9: -90.04,
  10: -89.17,
  11: -88.36},
 3: {0: -87.68,
  1: -88.29,
  2: -88.92,
  3: -89.47,
  4: -89.78,
  5: -89.96,
  6: -90.08,
  7: -90.01,
  8: -89.62,
  9: -88.97,
  10: -87.96,
  11: -86.98},
 4: {0: -86.21,
  1: -86.82,
  2: -87.41,
  3: -87.9,
  4: -88.14,
  5: -88.25,
  6: -88.33,
  7: -88.19,
  8: -87.61,
  9: -86.76,
  10: -85.66,
  11: -84.58},
 5: {0: -85.06,
  1: -85.66,
  2: -86.05,
  3: -86.27,
  4: -86.22,
  5: -86.04,
  6: -85.9,
  7: -85.58,
  8: -84.86,
  9: -83.88,
  10: -82.63,
  11: -81.36},
 6: {0: -84.07,
  1: -84.53,
  2: -84.53

In [44]:

print (f"the total time spent is {(time.time()-start_time):.0f} seconds")

the total time spent is 119 seconds


In [ ]:
from math import *

In [ ]:
def d(x0, y0, x1, y1):
	return sqrt((x0-x1)**2 + (y0-y1)**2)

In [ ]:
def g(alpha, instance):
	for i in range(len(instance.TS)-1):
		w_i = cos(instance.TS[i][0]/180.0*pi)
		w_ip1 = cos(instance.TS[i+1][0]/180.0*pi)
		s_i = instance.TS[i][1]
		s_ip1 = instance.TS[i+1][1]

		if w_i >= alpha and alpha >= w_ip1:
			return ( s_i + ( (s_ip1 - s_i) * (alpha - w_i) ) / ( w_ip1 - w_i ) )
		elif -w_i <= alpha and alpha <= -w_ip1:
			return ( s_i + ( (s_ip1 - s_i) * (alpha + w_i) ) / ( w_i - w_ip1 ) )

	return 0

In [ ]:
def check_line(x0, y0, x1, y1, map):
	dx = abs(x1-x0)
	if x0<x1:
		sx = 1
	else:
		sx = -1
	dy = -abs(y1-y0)
	if y0<y1:
		sy = 1
	else:
		sy = -1
	err = dx+dy

	while (1):
		if map[x0][y0] >= 0.0:
			return 1
		if x0==x1 and y0==y1:
			return None
		e2 = 2*err
		if e2 > dy:
			err = err + dy
			x0 = x0 + sx
		if e2 < dx:
			err = err + dx
			y0 = y0 + sy


In [ ]:
print ("Computing coverage")

detection_prob = {}

start_time_coverage = time.time()

if len(instance.TS) == 0: # without TS

	for tar_x, tar_y in ocean: # target

		for tx_x, tx_y in ocean: # source

			for rx_x, rx_y in ocean: # receiver

				# no obstacles between source-target and target-receiver, and source-reiver	
				if check_line(tx_x, tx_y, tar_x, tar_y, map) == None and check_line(tar_x, tar_y, rx_x, rx_y, map) == None:

					if instance.CC == 0: # probabilistic model
						
						if (((d(tx_x, tx_y, tar_x, tar_y) * d(rx_x, rx_y, tar_x, tar_y)) / (instance.rho_0**2) - 1) / instance.b1) < e+10: # avoid numerical trouble from powers of large numbers
							aux = instance.pmax * (1 / (1 + 10**(((d(tx_x, tx_y, tar_x, tar_y) * d(rx_x, rx_y, tar_x, tar_y)) / (instance.rho_0**2) - 1) / instance.b1))) * (1 / (1 + 10**((1 - (d(tx_x, tx_y, tar_x, tar_y) + d(rx_x, rx_y, tar_x, tar_y)) / (d(tx_x, tx_y, rx_x, rx_y) + 2*instance.rb)) / instance.b2)))
							
							if aux > instance.pmin:
								
								detection_prob[tar_x,tar_y,0,tx_x,tx_y,rx_x,rx_y] = log(1 - aux) # detection probabilitar_y

					else: # cookie-cutter model

						if d(tx_x, tx_y, tar_x, tar_y) * d(rx_x, rx_y, tar_x, tar_y) <= instance.rho_0**2 and d(tx_x, tx_y, tar_x, tar_y) + d(rx_x, rx_y, tar_x, tar_y) >= d(tx_x, tx_y, rx_x, rx_y) + 2*instance.rb: # check for inside range-of-day Cassini oval and outside direct-blast-effect
							
							detection_prob[tar_x, tar_y, 0, tx_x, tx_y, rx_x, rx_y] = 1 # sure detection

else: # with TS

	# only compute the angle we need to compute???

	for tar_x, tar_y in ocean: # target

		for tx_x, tx_y in ocean: # source

			if (tx_x,tx_y) != (tar_x,tar_y): # exclude equalitar_y of source and target (direct blast effect)
				
				for rx_x, rx_y in ocean: # receiver

					if (rx_x,rx_y) != (tar_x,tar_y): # exclude equalitar_y of receiver and target (direct blast effect)
					
						# no obstacles between source-target and target-receiver, and source-reiver
						if check_line(tx_x, tx_y, tar_x, tar_y, map) == None and check_line(tar_x, tar_y, rx_x, rx_y, map) == None:

							sqrt_tx_tar = 0.5 / ( sqrt((tx_x-tar_x)**2 + (tx_y-tar_y)**2) )
							sqrt_rx_tar = 0.5 / ( sqrt((rx_x-tar_x)**2 + (rx_y-tar_y)**2) )

							for theta in range(0, 180, instance.STEPS): # target angle

								my_theta = theta / 180.0 * pi
								my_sin_theta = sin(my_theta)
								my_cos_theta = cos(my_theta)

								if instance.CC == 0: # probabilistic model

									alpha = ( ((tx_x-tar_x)*my_cos_theta + (tx_y-tar_y)*my_sin_theta ) * sqrt_tx_tar + ((rx_x-tar_x)*my_cos_theta + (rx_y-tar_y)*my_sin_theta ) * sqrt_rx_tar )

									if (((d(tx_x,tx_y,tar_x,tar_y) * d(rx_x,rx_y,tar_x,tar_y)) / ((instance.rho_0 + g(alpha, instance))**2) - 1)/instance.b1) < e+10: # avoid numerical trouble from powers of large numbers
										
										aux = instance.pmax * (1 / (1 + 10**(((d(tx_x,tx_y,tar_x,tar_y) * d(rx_x,rx_y,tar_x,tar_y)) / ((instance.rho_0 + g(alpha, instance))**2) - 1)/instance.b1))) * (1 / (1 + 10**((1 - (d(tx_x,tx_y,tar_x,tar_y) + d(rx_x,rx_y,tar_x,tar_y)) / (d(tx_x,tx_y,rx_x,rx_y) + 2*instance.rb))/instance.b2)))
										
										if aux > instance.pmin:
											
											detection_prob[tar_x,tar_y,theta,tx_x,tx_y,rx_x,rx_y] = log(1 - aux) # detection probabilitar_y

								else: # cookie-cutter model
									
									if d(tx_x,tx_y,tar_x,tar_y) + d(rx_x,rx_y,tar_x,tar_y) >= d(tx_x,tx_y,rx_x,rx_y) + 2*instance.rb: # check for outside direct-blast-effect

										alpha = ( ((tx_x-tar_x) * my_cos_theta + (tx_y-tar_y) * my_sin_theta ) * sqrt_tx_tar + ((rx_x-tar_x) * my_cos_theta + (rx_y-tar_y) * my_sin_theta ) * sqrt_rx_tar )

										#print ("target:",tar_x,tar_y,"angle:",theta,"source:",tx_x,tx_y,"receiver:",rx_x,rx_y,"E-angle:",alpha*180/pi,"TS:",g_cos(alpha))

										if d(tx_x,tx_y,tar_x,tar_y) * d(rx_x,rx_y,tar_x,tar_y) <= (instance.rho_0 + g(alpha, instance))**2: # check for inside range-of-day Cassini oval

											detection_prob[tar_x,tar_y,theta,tx_x,tx_y,rx_x,rx_y] = 1 # sure detection

											#print ("target:",tar_x,tar_y,"angle:",theta,"source:",tx_x,tx_y,"receiver:",rx_x,rx_y,"E-angle:",alpha*180/pi,"TS:",g_cos(alpha))

end_time_coverage = time.time()

print (f"it took {(end_time_coverage - start_time_coverage):.2f} sec to get {len(detection_prob)} detection triples")